In [6]:
import pandas as pd
import math
import random
import numpy as np
from matplotlib import pyplot as plt

In [7]:
def relu(x): return x if x > 0 else 0.0
def relu_derivative(x): return 1.0 if x > 0 else 0.0
def sigmoid(t): return 1 / (1 + math.exp(-t))
def derivative_sigmoid(sigx): return (sigx) * (1 - sigx)
def dot(x, y):
    x = np.asarray(x)
    y = np.asarray(y)
    assert x.shape == y.shape, "Mismatch of shapes: " + str(x) + " " + str(y) + " " + str(x.shape) + " " + str(y.shape)
    assert x.ndim == 1, "Not a vector"
    #print("SUCCESS AT DOT PRODUCT BOi")
    return sum([m*n for m, n in zip(x, y)])

def feed_forward(network, inp, activation):
    outputs = list()
    inp2 = inp.copy()
    inp2.append(1)
    outputs.append(inp2)
    index = 0
    for layer in network:
        layer_list = list()
        for weight_set in layer:
            layer_list.append(activation(dot(weight_set, outputs[-1])))
        if index == len(network) - 1:
            outputs.append((layer_list))
        else:
            layer_list_copy = layer_list.copy()
            layer_list_copy.append(1)
            outputs.append((layer_list_copy))
        index += 1
                           
    return outputs

In [8]:
def sqerror_gradient(networks, inp, correct_output, activation, derivative):
    outputs = feed_forward(network, inp, activation)
    node_errors, weight_gradients = list(), list()
    for layer in range(len(outputs) - 1, -1, -1): # going backwards
        if layer == len(outputs) - 1: # the output layer
            currentLayerError = list()
            n = 0
            for node in outputs[layer]:
                assert n <= len(correct_output) - 1, print("BOI: ", correct_output, n, outputs)
                currentLayerError.append((node - correct_output[n]) * derivative(node))
                n += 1
            #print("ERROR last layer", currentLayerError)
            node_errors.append(currentLayerError)
            
            currentLayerGradient = list()
            output_node = 0
            for weight_set in networks[layer-1]:
                input_node = 0
                weight_set_gradient = list()
                for weights in weight_set:
                    weight_set_gradient.append(node_errors[-1][output_node] * outputs[layer-1][input_node])
                    input_node += 1
                output_node += 1
                currentLayerGradient.append(weight_set_gradient)
            weight_gradients.append(currentLayerGradient)
        
        elif layer >= 1:
            currentLayerError = list()
            n = 0
            for node in outputs[layer]:
                weight_sum = 0
                if n < len(outputs[layer]) - 1:
                    m = 0
                    for weight_set in networks[layer]:
                        weight_sum += weight_set[n] * node_errors[-1][m]
                        m += 1
                    currentLayerError.append(weight_sum * derivative(node))
                n += 1
            #print("ERROR layer %s: %s" % (layer, currentLayerError))
            node_errors.append(currentLayerError)
            
            currentLayerGradient = list()
            output_node = 0
            for weight_set in networks[layer-1]:
                input_node = 0
                weight_set_gradient = list()
                for weights in weight_set:
                    weight_set_gradient.append(node_errors[-1][output_node] * outputs[layer-1][input_node])
                    input_node += 1
                output_node += 1
                currentLayerGradient.append(weight_set_gradient)
            weight_gradients.append(currentLayerGradient)
    weight_gradients.reverse()
    return weight_gradients

In [13]:
def train(network, inp, out, activation, derivative, learning_rate = 0.1, epoch = 100):
    for i in range(epoch):
        for current_input, output in zip(inp, out):
            weight_gradients = sqerror_gradient(network, current_input, output, activation, derivative)
            for layer in range(len(network)):
                for node in range(len(network[layer])):
                    for weight in range(len(network[layer][node])):
                        network[layer][node][weight] -= learning_rate * weight_gradients[layer][node][weight]
        #print("EPOCH: %s" % (i,))
    return network
                    

In [14]:
def build_network(dimension, INPUT_NUM):
    layer_num = 0
    network = list()
    for num_nodes in dimension:
        layer = list()
        for i in range(num_nodes):
            if layer_num == 0:
                layer.append([random.random() for _ in range(INPUT_NUM)])
            else:
                layer.append([random.random() for _ in range(len(network[-1]) + 1)])
        network.append(layer)
        layer_num += 1
    return network

In [17]:
def testing(network, inp, out, activation):
    success = 0
    vector_round = np.vectorize(round)
    for a, b in zip(inp, out):
        if vector_round(feed_forward(network, a, activation)[-1]) == b: success += 1
    return (success / len(inp))

In [18]:
# TESTING FOR "AND" operator
inp = [[1, 1], [1, 0], [0, 1], [0, 0]]
out = [[1], [0], [0], [0]]
network = build_network([3, 2, 1], 3)
network = train(network, inp, out, sigmoid, derivative_sigmoid, 1, 500)
print("Accuracy: %s" % (testing(network, inp, out, sigmoid),))

Accuracy: 1.0


In [19]:
# TESTING FOR "OR" operator
inp = [[1, 1], [1, 0], [0, 1], [0, 0]]
out = [[1], [1], [1], [0]]
network = build_network([3, 2, 1], 3)
network = train(network, inp, out, sigmoid, derivative_sigmoid, 1, 500)
print("Accuracy: %s" % (testing(network, inp, out, sigmoid),))

Accuracy: 1.0


In [27]:
# TESTING FOR "XOR" operator
inp = [[1, 1], [1, 0], [0, 1], [0, 0]]
out = [[0], [1], [1], [0]]
network = build_network([3, 2, 1], 3)
network = train(network, inp, out, sigmoid, derivative_sigmoid, 1, 5000)
print("Accuracy: %s" % (testing(network, inp, out, sigmoid),))

Accuracy: 1.0


In [28]:
# TESTING FOR "NAND" operator
inp = [[1, 1], [1, 0], [0, 1], [0, 0]]
out = [[0], [1], [1], [1]]
network = build_network([3, 2, 1], 3)
network = train(network, inp, out, sigmoid, derivative_sigmoid, 1, 5000)
print("Accuracy: %s" % (testing(network, inp, out, sigmoid),))

Accuracy: 1.0


In [30]:
# TESTING FOR "NOR" operator
inp = [[1, 1], [1, 0], [0, 1], [0, 0]]
out = [[0], [0], [0], [1]]
network = build_network([3, 2, 1], 3)
network = train(network, inp, out, sigmoid, derivative_sigmoid, 1, 5000)
print("Accuracy: %s" % (testing(network, inp, out, sigmoid),))

Accuracy: 1.0


In [31]:
# TESTING FOR "XNOR" operator
inp = [[1, 1], [1, 0], [0, 1], [0, 0]]
out = [[1], [0], [0], [1]]
network = build_network([3, 2, 1], 3)
network = train(network, inp, out, sigmoid, derivative_sigmoid, 1, 5000)
print("Accuracy: %s" % (testing(network, inp, out, sigmoid),))

Accuracy: 1.0
